# Context Managers   <a href="https://colab.research.google.com/github/Ahmad-Zaki/Python-Notes/blob/main/Context%20Managers/context-managers.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

## Introduction

Context Managers allow you to allocate and release resources precisely when you want to. The most widely used example of context managers is the `with` statement. It lets you create a sprecific context for your code to run into, then remove the context once the code execution is done.

The `with` statement in Python is a quite useful tool for properly managing external resources in your programs. It allows you to take advantage of existing Context Managers to automatically handle the setup and teardown phases whenever you’re dealing with external resources or with operations that require those phases.

## Using a Context Manager

One of the most widely used examples of Context Managers is opening files:

In [1]:
with open("my_file.txt", "w") as my_file:
    my_file.write("Hello world!")

the previous code block does 3 things:
1. Sets up a context (the context here is that an opened file is assigned to a variable called `my_file`).
2. Run some code inside that context.
3. Removes the context by closing the file

If an error occurs while writing the data to the file, it tries to close it. The above code is equivalent to:

In [2]:
my_file = open('my_file.txt', 'w')
try:
    my_file.write('Hello world!')
except FileNotFoundError as e:
    print(f"Error occurred while writing to the file: {e}")
finally:
    my_file.close()

We can see the using `with` made the code much simpler by managing the opening and closing of the file. A common use case of Context Managers is locking and releasing resources, like what we did with closing an opened file.

## Managing Resources in Python

A common problem you’ll face in programming is how to properly manage external resources, such as files, network connections, ... etc. Sometimes, a program will retain those resources forever, even if you no longer need them. This will reduce the available memory every time we create or open a new instance of a given resource withour closing na existing one.

Managing external resources requires buikding a **setup** phase (where you access that resource) and **teardown** phase (where you release that resource). The teardown phase includes some cleanup actions like closing a file and disconnention and network connection. If we don't do this, the application may keep the resource alive, consuming our system resources like memory or bandwidth.

## Building a context manager

There are two ways to create our own Context Managers:
1. Class-based
2. Function-based

### Class-based Context Managers

The `with` statement in Python can be written in this general form:
```python
with expression as target_var:
    do_something(target_var)
```

the Context Manager here will be an object that results from executing the `expression`. In order for that object to function properly as a Context Manager, it must implement the **context management protocol**, which consists of 2 magic methods:
- `__enter__`: This is called by the `with` statement to enter the context.
- `__exit__`: This is called once the code reaches the end of the `with` block to deconstruct the context. 

With that in mind, Lets make our own file-opening Context Manager to see how it works:

In [9]:
class OpenFile:
    def __init__(self, file_name, mode):
        self.file_obj = open(file_name, mode)
    def __enter__(self):
        return self.file_obj
    def __exit__(self, type, value, traceback):
        self.file_obj.close()

Note that the `__exit__` method takes three arguments, which are required by every `__exit__` method in a Context Manager class. If an exception occurs, Python passes the `type`, `value` and `traceback` of the exception to the `__exit__` method. It allows the `__exit__` method to decide how to close the file and if any further steps are required. In our case we are simply them.

Now, we can try our Context Manager to see if it works as intended:

In [10]:
with OpenFile('my_file.txt', 'w') as my_file:
    my_file.write('This has been writting through my own context manager!')

Here's how Python deals with a `with` statement when it runs into it:
1. The expression `OpenFile('my_file.txt', 'w')` is executed to obtain a Context Manager object.
2. Calls `__enter__()` method on the Context Manager and allocates its return value to `my_file`.
3. Executes the `with` statement code block.
4. Calls `__exit__()` on the Context Manager when the `with` code block is done, even if an exception is raised inside it.

The `as` keyword is optional. Some context managers return `None` from `__enter__()` because they have no useful object to give back to the caller. In these cases, using `as` to allocate `None` to variable makes no sense.

### Function-based Context Managers.

Python’s generator functions and the `contextlib.contextmanager` decorator provide an alternative and convenient way to implement the context management protocol.

Consider the following Context Manager, which changes the current working directory for some code and then changes it back to the original directory:

In [11]:
from contextlib import contextmanager
import os

@contextmanager
def cwd(path):
    print("Setting up context...")
    old_dir = os.getcwd()
    os.chdir(path)

    yield

    print("removing context...")
    os.chdir(old_dir)

In [20]:
with cwd(".."):
    dirs = os.listdir()[1:]
    print(dirs)

Setting up context...
['Context Managers', 'Function Decorators', 'Iterators and Generators', 'readme.md', 'String Formatting', 'Timing and Profiling Python code']
removing context...


In this example, we can see two visible sections in `cwd()`. Before the `yield` statement, we have the setup section. where we place the code that acquires the managed resources. Everything before the yield runs when the flow of execution enters the context.

The `yield` statement provides the object that will be assigned to the with target variable. In the example above, we don't require the Context Manager to return anything.

After the `yield` statement, we have the teardown section, in which we can release the resources and do the cleanup. The code after `yield` runs at the end of the `with` block.

The `@contextmanager` decorator reduces the boilerplate required to create a context manager. Instead of writing a whole class with `__enter__` and `__exit__` methods, you just need to implement a generator function with a single `yield` that produces whatever you want `__enter__` to return.

This implementation and the one that uses the context management protocol are practically equivalent. Depending on which one you find more readable, you might prefer one over the other.

## Handling Exceptions

We have seen in the class-based Context Managers that `__exit__` accepts the `type`, `value` and `traceback` of the exception. This allows it to handle the exception properly or raise it if it can't handle it.

Let's get back to the `File` context manager we created before:
```python
class OpenFile:
    def __init__(self, file_name, mode):
        self.file_obj = open(file_name, mode)
    def __enter__(self):
        return self.file_obj
    def __exit__(self, type, value, traceback):
        self.file_obj.close()
```

What if our file object raises an exception? We might be trying to access a method on the file object which it does not supports. For instance:

In [21]:
with OpenFile("my_file.txt", "w") as my_file:
    my_file.whatever("What is this?!")

AttributeError: '_io.TextIOWrapper' object has no attribute 'whatever'

Here are the steps which are taken by the `with` statement when an error is encountered:
1. It passes the type, value and traceback of the error to the `__exit__` method.
2. the `__exit__` method tries to handle the exception.
3. `__exit__` returns `True` if the exception is handled successfully.
4. If anything other than `True` is returned by `__exit__` then the exception is raised by the `with` statement. 

In our example, `__exit__` returns `None`. Therefore, the `with` statement raises the exception. Let's modify it to supress the exception instead:

In [31]:
class OpenFile:
    def __init__(self, file_name, mode):
        self.file_obj = open(file_name, mode)
    def __enter__(self):
        return self.file_obj
    def __exit__(self, type, value, traceback):
        if type:
            print("Exception encounted!")
            print(f"\t Traceback: {traceback}")
            print(f"\t Type: {type}, Value: {value}")
            print("This exception will be ignored.")
        self.file_obj.close()
        return True

In [32]:
with OpenFile("my_file.txt", "w") as my_file:
    my_file.whatever("What is this?!")

Exception encounted!
	 Traceback: <traceback object at 0x000001D2CA6BF540>
	 Type: <class 'AttributeError'>, Value: '_io.TextIOWrapper' object has no attribute 'whatever'
This exception will be ignored.


As for the function-based Context Managers, handling exceptions is as simple as using a `try - except - finally` code block:

In [36]:
@contextmanager
def open_file(name, mode):
    f = open(name, mode)

    try:
        yield f

    except Exception as e:
        print(f"An exception occurred!\n {e}")
        
    finally:
        f.close()

In [37]:
with open_file("my_file.txt", "w") as my_file:
    my_file.whatever("What is this?!")

An exception occurred!
 '_io.TextIOWrapper' object has no attribute 'whatever'
